# How to create a source

This tutorial demonstrates how to create a source.

There are different type of sources available: luminaire source, surface source, ray file source.

## Prerequisites

### Perform imports

In [1]:
from pathlib import Path

from ansys.speos.core import GeoRef, Project, Speos, launcher
from ansys.speos.core.kernel.client import (
    default_docker_channel,
)
from ansys.speos.core.source import (
    SourceAmbientEnvironment,
    SourceAmbientNaturalLight,
    SourceLuminaire,
    SourceRayFile,
    SourceSurface,
)


### Define constants
Constants help ensure consistency and avoid repetition throughout the example.

In [2]:
HOSTNAME = "localhost"
GRPC_PORT = 50098  # Be sure the Speos GRPC Server has been started on this port.
USE_DOCKER = True  # Set to False if you're running this example locally as a Notebook.
IES = "IES_C_DETECTOR.ies"

### Define helper functions

In [3]:
def create_helper_geometries(project: Project):
    """Create bodies and faces."""

    def create_face(body):
        (
            body.create_face(name="TheFaceF")
            .set_vertices([0, 0, 0, 1, 0, 0, 0, 1, 0])
            .set_facets([0, 1, 2])
            .set_normals([0, 0, 1, 0, 0, 1, 0, 0, 1])
            .commit()
        )

    root_part = project.create_root_part().commit()
    body_b1 = root_part.create_body(name="TheBodyB").commit()
    body_b2 = root_part.create_body(name="TheBodyC").commit()
    body_b3 = root_part.create_body(name="TheBodyD").commit()
    body_b4 = root_part.create_body(name="TheBodyE").commit()
    for b in [body_b1, body_b2, body_b3, body_b4]:
        create_face(b)

## Model Setup

### Load assets
The assets used to run this example are available in the
[PySpeos repository](https://github.com/ansys/pyspeos/) on GitHub.

> **Note:** Make sure you
> have downloaded simulation assets and set ``assets_data_path``
> to point to the assets folder.

In [4]:
if USE_DOCKER:  # Running on the remote server.
    assets_data_path = Path("/app") / "assets"
else:
    assets_data_path = Path("/path/to/your/download/assets/directory")

### Connect to the RPC Server
This Python client connects to a server where the Speos engine
is running as a service. In this example, the server and
client are the same machine. The launch_local_speos_rpc_method can
be used to start a local instance of the service.

In [5]:
if USE_DOCKER:
    speos = Speos(channel=default_docker_channel())
else:
    speos = launcher.launch_local_speos_rpc_server(port=GRPC_PORT)

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.13/site-packages/ansys/tools/common/cyberchannel.py:188: UserWarning: Starting gRPC client without TLS on localhost:50098. This is INSECURE. Consider using a secure connection.
  warn(f"Starting gRPC client without TLS on {target}. This is INSECURE. Consider using a secure connection.")


### Create a new project

The only way to create a source using the core layer, is to create it from a project.
The ``Project`` class is instantiated by passing a ``Speos`` instance

In [6]:
p = Project(speos=speos)
print(p)

{
    "name": "",
    "description": "",
    "metadata": {},
    "part_guid": "",
    "sources": [],
    "sensors": [],
    "simulations": [],
    "materials": [],
    "scenes": []
}


### Source Creation

**Create locally:**
The mention "local: " is added when printing the source data and information is not yet
pushed to the RPC server

In [7]:
intensity_file_path = str(assets_data_path / IES)
source1 = p.create_source(name="Luminaire.1", feature_type=SourceLuminaire)  # type luminaire
source1.set_intensity_file_uri(uri=intensity_file_path)
print(source1)

local: {
    "name": "Luminaire.1",
    "description": "",
    "metadata": {},
    "source_guid": "",
    "source": {
        "name": "Luminaire.1",
        "luminaire": {
            "flux_from_intensity_file": {},
            "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
            "spectrum_guid": "",
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ]
        },
        "description": "",
        "metadata": {}
    }
}


/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.13/site-packages/ansys/speos/core/project.py:230: UserWarning: The pySpeos feature : SourceLuminaire needs a Speos Version of 2025 R2 SP0 or higher.
  feature = SourceLuminaire(


**Push it to the server.**

After it is committed to the server, the mention "local: " is no more present when printing the
source.

In [8]:
source1.commit()
print(source1)

{
    "name": "Luminaire.1",
    "metadata": {
        "UniqueId": "20cdbb02-7237-4f77-a9e9-88f12321b962"
    },
    "source_guid": "cd3267b7-78f5-447d-ba5d-3f64a3f72f61",
    "description": "",
    "source": {
        "name": "Luminaire.1",
        "luminaire": {
            "flux_from_intensity_file": {},
            "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
            "spectrum_guid": "bd0aeb6c-702a-46d7-9d26-c4ffa4fb680c",
            "spectrum": {
                "name": "Luminaire.1.Spectrum",
                "predefined": {
                    "incandescent": {}
                },
                "description": "",
                "metadata": {}
            },
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ]
        },
    

**Changing additional Source Properties**

Setting several more characteristics.

In [9]:
intensity_file_path = str(assets_data_path / IES)
source2 = p.create_source(name="Luminaire.2", feature_type=SourceLuminaire)
source2.set_intensity_file_uri(uri=intensity_file_path)
source2.set_flux_radiant()  # select flux radiant with default value
# choose the source location [Origin, Xvector, Yvector, Zvector]
source2.set_axis_system(axis_system=[20, 50, 10, 1, 0, 0, 0, 1, 0, 0, 0, 1])
source2.set_spectrum().set_blackbody()  # choose blackbody with default value for the spectrum
source2.commit()  # Push to the server
print(source2)

{
    "name": "Luminaire.2",
    "metadata": {
        "UniqueId": "2133cda2-153c-4f11-ae89-19c2a794b77d"
    },
    "source_guid": "0a6b7d69-5f60-4f21-81c7-ec7c8ecfe62f",
    "description": "",
    "source": {
        "name": "Luminaire.2",
        "luminaire": {
            "radiant_flux": {
                "radiant_value": 1.0
            },
            "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
            "spectrum_guid": "8aa9f429-ed0f-4410-a4b6-c6bd34302359",
            "spectrum": {
                "name": "Luminaire.2.Spectrum",
                "blackbody": {
                    "temperature": 2856.0
                },
                "description": "",
                "metadata": {}
            },
            "axis_system": [
                20.0,
                50.0,
                10.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
      

**Source Instance**

As mention "local: " is added if it is not yet committed to the server.

In [10]:
print(source1)

{
    "name": "Luminaire.1",
    "metadata": {
        "UniqueId": "20cdbb02-7237-4f77-a9e9-88f12321b962"
    },
    "source_guid": "cd3267b7-78f5-447d-ba5d-3f64a3f72f61",
    "description": "",
    "source": {
        "name": "Luminaire.1",
        "luminaire": {
            "flux_from_intensity_file": {},
            "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
            "spectrum_guid": "bd0aeb6c-702a-46d7-9d26-c4ffa4fb680c",
            "spectrum": {
                "name": "Luminaire.1.Spectrum",
                "predefined": {
                    "incandescent": {}
                },
                "description": "",
                "metadata": {}
            },
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ]
        },
    

**Project:**

Committed feature will appear inside the project information.

In [11]:
print(p)

{
    "sources": [
        {
            "name": "Luminaire.1",
            "metadata": {
                "UniqueId": "20cdbb02-7237-4f77-a9e9-88f12321b962"
            },
            "source_guid": "cd3267b7-78f5-447d-ba5d-3f64a3f72f61",
            "description": "",
            "source": {
                "name": "Luminaire.1",
                "luminaire": {
                    "flux_from_intensity_file": {},
                    "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
                    "spectrum_guid": "bd0aeb6c-702a-46d7-9d26-c4ffa4fb680c",
                    "spectrum": {
                        "name": "Luminaire.1.Spectrum",
                        "predefined": {
                            "incandescent": {}
                        },
                        "description": "",
                        "metadata": {}
                    },
                    "axis_system": [
                        0.0,
                        0.0,
                        0.0

**Update:**

> **Note:** If you are manipulating a source already committed, don't forget to commit your
> changes.
> If you don't, you will still only watch what is committed on the server.

In [12]:
source1.set_flux_radiant(value=1.2)  # modify radiant flux value
source1.set_axis_system(axis_system=[17, 10, 10, 1, 0, 0, 0, 1, 0, 0, 0, 1])  # modify axis system
source1.set_spectrum().set_halogen()  # modify spectrum by choosing halogen
source1.commit()  # Push changes to the server
print(source1)

{
    "name": "Luminaire.1",
    "metadata": {
        "UniqueId": "20cdbb02-7237-4f77-a9e9-88f12321b962"
    },
    "source_guid": "cd3267b7-78f5-447d-ba5d-3f64a3f72f61",
    "description": "",
    "source": {
        "name": "Luminaire.1",
        "luminaire": {
            "radiant_flux": {
                "radiant_value": 1.2
            },
            "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
            "spectrum_guid": "bd0aeb6c-702a-46d7-9d26-c4ffa4fb680c",
            "spectrum": {
                "name": "Luminaire.1.Spectrum",
                "predefined": {
                    "halogen": {}
                },
                "description": "",
                "metadata": {}
            },
            "axis_system": [
                17.0,
                10.0,
                10.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
             

**Reset**

Possibility to reset local values from the one available in the server.

In [13]:
source1.set_flux_luminous()  # modify to luminous flux BUT no commit
source1.reset()
# reset -> this will apply the server value to the local value the local value will be back to
# halogen
source1.delete()  # delete (to display the local value with the below print)
print(source1)

local: {
    "name": "Luminaire.1",
    "description": "",
    "metadata": {},
    "source_guid": "",
    "source": {
        "name": "Luminaire.1",
        "luminaire": {
            "radiant_flux": {
                "radiant_value": 1.2
            },
            "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
            "spectrum_guid": "bd0aeb6c-702a-46d7-9d26-c4ffa4fb680c",
            "spectrum": {
                "name": "Luminaire.1.Spectrum",
                "predefined": {
                    "halogen": {}
                },
                "description": "",
                "metadata": {}
            },
            "axis_system": [
                17.0,
                10.0,
                10.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ]
        },
        "description": "",
        "metadata": {}
    }
}


**Delete**

Once the data is deleted from the server, you can still work with local data and maybe commit
later.

In [14]:
source2.delete()
print(source2)
source1.delete()
print(p)

local: {
    "name": "Luminaire.2",
    "description": "",
    "metadata": {},
    "source_guid": "",
    "source": {
        "name": "Luminaire.2",
        "luminaire": {
            "radiant_flux": {
                "radiant_value": 1.0
            },
            "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
            "spectrum_guid": "8aa9f429-ed0f-4410-a4b6-c6bd34302359",
            "spectrum": {
                "name": "Luminaire.2.Spectrum",
                "blackbody": {
                    "temperature": 2856.0
                },
                "description": "",
                "metadata": {}
            },
            "axis_system": [
                20.0,
                50.0,
                10.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ]
        },
        "description": "",
        "metadata": {}
    

## Other Sources Examples

### Ray-file source

In [15]:
ray_file_path = str(assets_data_path / "Rays.ray")

source3 = p.create_source(name="Ray-file.1", feature_type=SourceRayFile)  # type ray file
source3.set_ray_file_uri(uri=ray_file_path)
source3.commit()
print(source3)

{
    "name": "Ray-file.1",
    "metadata": {
        "UniqueId": "b22988df-cf1a-490c-a31f-5b671ca88501"
    },
    "source_guid": "dfeae674-3e4a-4b46-b7c3-9fb41c2c3237",
    "description": "",
    "source": {
        "name": "Ray-file.1",
        "rayfile": {
            "ray_file_uri": "/app/assets/Rays.ray",
            "flux_from_ray_file": {},
            "spectrum_from_ray_file": {},
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ]
        },
        "description": "",
        "metadata": {}
    }
}


/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.13/site-packages/ansys/speos/core/project.py:223: UserWarning: The pySpeos feature : SourceRayFile needs a Speos Version of 2025 R2 SP0 or higher.
  feature = SourceRayFile(


In [16]:
source3.set_flux_luminous()
source3.commit()
print(source3)

{
    "name": "Ray-file.1",
    "metadata": {
        "UniqueId": "b22988df-cf1a-490c-a31f-5b671ca88501"
    },
    "source_guid": "dfeae674-3e4a-4b46-b7c3-9fb41c2c3237",
    "description": "",
    "source": {
        "name": "Ray-file.1",
        "rayfile": {
            "ray_file_uri": "/app/assets/Rays.ray",
            "luminous_flux": {
                "luminous_value": 683.0
            },
            "spectrum_from_ray_file": {},
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ]
        },
        "description": "",
        "metadata": {}
    }
}


In [17]:
source3.delete()

### Surface source

In [18]:
create_helper_geometries(p)
source4 = p.create_source(name="Surface.1", feature_type=SourceSurface)
source4.set_exitance_constant(
    geometries=[
        (GeoRef.from_native_link("TheBodyB/TheFaceF"), False),
        (GeoRef.from_native_link("TheBodyC/TheFaceF"), True),
    ]
)
source4.commit()
print(source4)

{
    "name": "Surface.1",
    "metadata": {
        "UniqueId": "04b969f9-e749-4707-9101-71d5b80c762e"
    },
    "source_guid": "f815d029-2cbe-49fb-891e-58984232e3ec",
    "description": "",
    "source": {
        "name": "Surface.1",
        "surface": {
            "luminous_flux": {
                "luminous_value": 683.0
            },
            "intensity_guid": "f4cf2867-efe9-4da5-93e4-ff0225ac11d6",
            "exitance_constant": {
                "geo_paths": [
                    {
                        "geo_path": "TheBodyB/TheFaceF",
                        "reverse_normal": false
                    },
                    {
                        "geo_path": "TheBodyC/TheFaceF",
                        "reverse_normal": true
                    }
                ]
            },
            "spectrum_guid": "af0b303a-ef7f-496a-8431-a5a5b313608c",
            "intensity": {
                "name": "Surface.1.Intensity",
                "cos": {
                    

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.13/site-packages/ansys/speos/core/project.py:216: UserWarning: The pySpeos feature : SourceSurface needs a Speos Version of 2025 R2 SP0 or higher.
  feature = SourceSurface(


In [19]:
source4.set_flux_luminous_intensity()
source4.set_intensity().set_gaussian().set_axis_system(
    axis_system=[10, 50, 20, 1, 0, 0, 0, 1, 0, 0, 0, 1]
)
source4.commit()
print(source4)

{
    "name": "Surface.1",
    "metadata": {
        "UniqueId": "04b969f9-e749-4707-9101-71d5b80c762e"
    },
    "source_guid": "f815d029-2cbe-49fb-891e-58984232e3ec",
    "description": "",
    "source": {
        "name": "Surface.1",
        "surface": {
            "luminous_intensity_flux": {
                "luminous_intensity_value": 5.0
            },
            "intensity_guid": "f4cf2867-efe9-4da5-93e4-ff0225ac11d6",
            "exitance_constant": {
                "geo_paths": [
                    {
                        "geo_path": "TheBodyB/TheFaceF",
                        "reverse_normal": false
                    },
                    {
                        "geo_path": "TheBodyC/TheFaceF",
                        "reverse_normal": true
                    }
                ]
            },
            "spectrum_guid": "af0b303a-ef7f-496a-8431-a5a5b313608c",
            "intensity": {
                "name": "Surface.1.Intensity",
                "gaussian":

In [20]:
source4.delete()
print(source4)

local: {
    "name": "Surface.1",
    "description": "",
    "metadata": {},
    "source_guid": "",
    "source": {
        "name": "Surface.1",
        "surface": {
            "luminous_intensity_flux": {
                "luminous_intensity_value": 5.0
            },
            "intensity_guid": "f4cf2867-efe9-4da5-93e4-ff0225ac11d6",
            "exitance_constant": {
                "geo_paths": [
                    {
                        "geo_path": "TheBodyB/TheFaceF",
                        "reverse_normal": false
                    },
                    {
                        "geo_path": "TheBodyC/TheFaceF",
                        "reverse_normal": true
                    }
                ]
            },
            "spectrum_guid": "af0b303a-ef7f-496a-8431-a5a5b313608c",
            "intensity": {
                "name": "Surface.1.Intensity",
                "gaussian": {
                    "FWHM_angle_x": 30.0,
                    "FWHM_angle_y": 30.0,
      

### Ambient natural light source

In [21]:
source5 = p.create_source(name="NaturalLight.1", feature_type=SourceAmbientNaturalLight)
source5.turbidity = 4
source5.with_sky = True
print(source5.zenith_direction)  # default zenith direction
print(source5.north_direction)  # default north direction
source5.reverse_north_direction = True
print(source5)

source5.commit()
print(source5)

[0.0, 0.0, 1.0]
[0.0, 1.0, 0.0]
local: {
    "name": "NaturalLight.1",
    "description": "",
    "metadata": {},
    "source_guid": "",
    "source": {
        "name": "NaturalLight.1",
        "ambient": {
            "natural_light": {
                "turbidity": 4.0,
                "with_sky": true,
                "north_direction": [
                    0.0,
                    1.0,
                    0.0
                ],
                "reverse_north": true,
                "sun_axis_system": {
                    "automatic_sun": {
                        "time_zone_uri": "CET",
                        "year": 2026,
                        "month": 2,
                        "day": 25,
                        "hour": 13,
                        "minute": 31,
                        "longitude": 0.0,
                        "latitude": 0.0
                    }
                }
            },
            "zenith_direction": [
                0.0,
                0.0,
    

{
    "name": "NaturalLight.1",
    "metadata": {
        "UniqueId": "34d7b7fc-eb73-489b-aded-85f682b36689"
    },
    "source_guid": "3715b916-ce8b-4ecc-a291-f11ef952ff83",
    "description": "",
    "source": {
        "name": "NaturalLight.1",
        "ambient": {
            "natural_light": {
                "turbidity": 4.0,
                "with_sky": true,
                "north_direction": [
                    0.0,
                    1.0,
                    0.0
                ],
                "reverse_north": true,
                "sun_axis_system": {
                    "automatic_sun": {
                        "time_zone_uri": "CET",
                        "year": 2026,
                        "month": 2,
                        "day": 25,
                        "hour": 13,
                        "minute": 31,
                        "longitude": 0.0,
                        "latitude": 0.0
                    }
                }
            },
            "zenith

In [22]:
source5.set_sun_automatic().year = 2026
source5.set_sun_automatic().month = 12
source5.set_sun_automatic().day = 31
source5.set_sun_automatic().hour = 12
source5.set_sun_automatic().minute = 23
source5.set_sun_automatic().longitude = 10
source5.set_sun_automatic().latitude = 45
source5.set_sun_automatic().time_zone = "CST"
source5.commit()
print(source5)

{
    "name": "NaturalLight.1",
    "metadata": {
        "UniqueId": "34d7b7fc-eb73-489b-aded-85f682b36689"
    },
    "source_guid": "3715b916-ce8b-4ecc-a291-f11ef952ff83",
    "description": "",
    "source": {
        "name": "NaturalLight.1",
        "ambient": {
            "natural_light": {
                "turbidity": 4.0,
                "with_sky": true,
                "north_direction": [
                    0.0,
                    1.0,
                    0.0
                ],
                "reverse_north": true,
                "sun_axis_system": {
                    "automatic_sun": {
                        "time_zone_uri": "CST",
                        "year": 2026,
                        "month": 12,
                        "day": 31,
                        "hour": 12,
                        "minute": 23,
                        "longitude": 10.0,
                        "latitude": 45.0
                    }
                }
            },
            "zen

In [23]:
source5.delete()

### Ambient environment light source

In [24]:
source6 = p.create_source(name="Environment.1", feature_type=SourceAmbientEnvironment)
image_file_uri = str(assets_data_path / "stars.exr")
source6.image_file_uri = image_file_uri
print(source6.zenith_direction)  # default zenith direction
print(source6.north_direction)  # default north direction
source6.reverse_north_direction = True
print(source6.luminance)  # default luminance value
source6.set_predefined_color_space().set_color_space_adobergb()
source6.set_predefined_color_space().set_color_space_srgb()
print(source6.color_space)
source6.set_userdefined_color_space().set_white_point_type_d50()
print(source6.set_userdefined_color_space().white_point_type)
source6.set_userdefined_color_space().red_spectrum = str(
    assets_data_path / "LG_50M_Colorimetric_short.sv5" / "Red Spectrum.spectrum"
)
source6.set_userdefined_color_space().blue_spectrum = str(
    assets_data_path / "LG_50M_Colorimetric_short.sv5" / "Blue Spectrum.spectrum"
)
source6.set_userdefined_color_space().green_spectrum = str(
    assets_data_path / "LG_50M_Colorimetric_short.sv5" / "Blue Spectrum.spectrum"
)
source6.commit()
print(source6)

[0.0, 0.0, 1.0]
[0.0, 1.0, 0.0]
1000.0
1


{
    "name": "Environment.1",
    "metadata": {
        "UniqueId": "93a432cc-1807-437a-bb43-b52b65760081"
    },
    "source_guid": "ade37007-8939-40b9-be43-bd01d7f1fec9",
    "description": "",
    "source": {
        "name": "Environment.1",
        "ambient": {
            "environment_map": {
                "luminance": 1000.0,
                "image_uri": "/app/assets/stars.exr",
                "user_defined_rgb_space": {
                    "pre_defined_white_point": {
                        "white_point_type": "D50"
                    },
                    "red_spectrum_guid": "e45e0fe5-0417-452f-bb61-434067297f54",
                    "green_spectrum_guid": "0f4104b9-8728-4594-8ab6-65b81f574be8",
                    "blue_spectrum_guid": "496f85f8-468d-4163-a1f9-3ff329cfa159",
                    "red_spectrum": {
                        "name": "Red Spectrum.spectrum",
                        "library": {
                            "file_uri": "/app/assets/LG_50M_Color

In [25]:
ground_plane = (
    p.create_ground_plane()
)  #  Only usable when there is at least one Ambient Environment Source
ground_plane.ground_zenith = [0, 0, 1]
ground_plane.ground_origin = [10, 0, 0]
ground_plane.ground_height = 1000
ground_plane.commit()
print(p)

ground_plane2 = p.create_ground_plane()
ground_plane2.ground_height = 2000
ground_plane2.commit()
print(p)  # only 1 ground plane is taken per project, the second overwrites the previous one.

{
    "part_guid": "d3b44617-87bb-4504-9a3f-31146311fdc7",
    "sources": [
        {
            "name": "Environment.1",
            "metadata": {
                "UniqueId": "93a432cc-1807-437a-bb43-b52b65760081"
            },
            "source_guid": "ade37007-8939-40b9-be43-bd01d7f1fec9",
            "description": "",
            "source": {
                "name": "Environment.1",
                "ambient": {
                    "environment_map": {
                        "luminance": 1000.0,
                        "image_uri": "/app/assets/stars.exr",
                        "user_defined_rgb_space": {
                            "pre_defined_white_point": {
                                "white_point_type": "D50"
                            },
                            "red_spectrum_guid": "e45e0fe5-0417-452f-bb61-434067297f54",
                            "green_spectrum_guid": "0f4104b9-8728-4594-8ab6-65b81f574be8",
                            "blue_spectrum_guid": "

{
    "part_guid": "d3b44617-87bb-4504-9a3f-31146311fdc7",
    "sources": [
        {
            "name": "Environment.1",
            "metadata": {
                "UniqueId": "93a432cc-1807-437a-bb43-b52b65760081"
            },
            "source_guid": "ade37007-8939-40b9-be43-bd01d7f1fec9",
            "description": "",
            "source": {
                "name": "Environment.1",
                "ambient": {
                    "environment_map": {
                        "luminance": 1000.0,
                        "image_uri": "/app/assets/stars.exr",
                        "user_defined_rgb_space": {
                            "pre_defined_white_point": {
                                "white_point_type": "D50"
                            },
                            "red_spectrum_guid": "e45e0fe5-0417-452f-bb61-434067297f54",
                            "green_spectrum_guid": "0f4104b9-8728-4594-8ab6-65b81f574be8",
                            "blue_spectrum_guid": "

In [26]:
ground_plane.delete()
source6.delete()

When creating sources, this creates some intermediate objects (spectrums, intensity templates).

Deleting a source does not delete in cascade those objects
because they could be used by some other entities from core layer.

Then at the end of the example, we just clean all databases

In [27]:
for item in speos.client.intensity_templates().list() + speos.client.spectrums().list():
    item.delete()

speos.close()

True